# Data-Driven Decks



## Inkscape Driver

> TBD

## Drawio Driver

In [ ]:
import base64, html, urllib.parse, pathlib as P, tempfile, asyncio, IPython
import IPython.display as D, networkx as nx, jinja2, babel.support as B, pandas as pd
import PyPDF2, traitlets as T, ipywidgets as W, tornado.ioloop, lxml.etree as E
from nbconvert.filters.markdown_mistune import markdown2html_mistune
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor
import requests, requests_cache

In [ ]:
requests_cache.install_cache("ddd", allowable_methods=["GET", "POST"])

## Slides

In [ ]:
class Slides(W.HTML):
    """ some number of slides, as PDF
    """
    pdf = T.Unicode()
    
    def __init__(self, *args, **kwargs):
        if not kwargs.get("layout"):
            kwargs["layout"] = dict(
                display = "flex",
                flex_flow = "column wrap",
                flex = "1",
                height = "100%",
            )
        super().__init__(*args, **kwargs)
    
    @T.observe("pdf")
    def _on_pdf(self, change):
        self.value = f"""
            <iframe 
                src="data:application/pdf;base64,{change.new}" 
                style="border: 0; min-width: 400px; min-height: 400px; width: 100%; height: 100%;">
            </iframe>
        """

### DrawioSlides

### Before we begin

> ## This is **NOT READY** for prime-time
> Start the **demo** export server. It will try to install its dependencies with `jlpm`, and requires `nodejs`.
> ```bash
> !python scripts/drawio_export_demo.py
> ```

In [ ]:
class DrawioSlides(Slides):
    """ Slides built with drawio-export from drawio XML
    """
    executor = ThreadPoolExecutor(max_workers=1)
    
    xml = T.Unicode()
    params = T.Dict()
    url = T.Unicode(default_value="http://localhost:8000")
    
    CORE_PARAMS = dict(
        format="pdf",
        base64="1"
    )
    
    @run_on_executor
    def update_pdf(self):
        # this really needs to be a queue
        self.value = "<blockquote>rendering...</blockqoute>"
        xml = self.transform(self.xml)
        r = requests.post(
            self.url, 
            timeout=None,
            data=dict(
                xml=xml, 
                **self.params, 
                **self.CORE_PARAMS
            ))
        self.pdf = r.text
    
    def transform(self, xml):
        return xml

    @T.observe("xml")
    def _on_xml(self, change=None):
        tornado.ioloop.IOLoop.current().add_callback(self.update_pdf)

In [ ]:
HOW_IT_WORKS = P.Path("testfiles/How it works.dio")
how_it_works = DrawioSlides(xml=HOW_IT_WORKS.read_text())
how_it_works

#### TemplatedDrawioSlides

In [ ]:
class TemplatedDrawioSlides(DrawioSlides):
    context = T.Dict()
    
    AMP = "&"
    _AMP_ = "_____AMP_____"

    def clean(self, txt):
        return txt.replace(self.AMP, self._AMP_)

    def smudge(self, txt):
        return txt.replace(self._AMP_, self.AMP)
    
    def transform(self, xml):
        env = jinja2.Environment(
          extensions=['jinja2.ext.i18n', 'jinja2.ext.autoescape'],
          autoescape=jinja2.select_autoescape(['html', 'xml'])
        )

        return self.smudge(env.from_string(self.clean(xml)).render(**{
            key: self.markdown(value)
            for key, value in (self.context or {}).items()
        }))
    
    @T.observe("context")
    def _on_context(self, change):
        self._on_xml()
    
    def markdown(self, md):
        return markdown2html_mistune(md).replace(self.AMP, self._AMP_)

In [ ]:
TEMPLATE = P.Path("testfiles/template deck.dio") 
title = TemplatedDrawioSlides(xml=TEMPLATE.read_text(), context=dict(
    hero="<h1>???</h1>",
    title="_No title here yet..._"
))
title

In [ ]:
logo = """<img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Jupyter_logo.svg" width="200" height="200"></image>"""

In [ ]:
title.context = {
    "title": "The _title_ can contain `Markdown`",
    "hero": logo
}

In [ ]:
ideas = []
for i in range(4):
    ideas += [
        TemplatedDrawioSlides(xml=TEMPLATE.read_text(), params=dict(math="0"), context=dict(
            title=f"# Idea {i + 1}",
            abstract=f"This is idea {i + 1}. It's better than [idea {i}](#idea-{i})",
            hero=(logo + "\n\n") * (1 + 1) 
        ))
    ]

## Deck

### Deck MK1 Prototype

The simplest deck prototype is just a box.

In [ ]:
deck = W.HBox([title, how_it_works, *ideas], 
              layout=dict(display="flex", flex_flow="row wrap"))
# deck

### Deck MK2 Prototype

This builds a composite deck.

In [ ]:
class Deck(W.HBox):
    composite = T.Unicode()
    preview = T.Instance(Slides)
    
    def __init__(self, *args, **kwargs):
        if "layout" not in kwargs:
            kwargs["layout"] = dict(display="flex", flex_flow="row wrap")
        super().__init__(*args, **kwargs)
    
    @T.default("preview")
    def _default_preview(self):
        slides = Slides()
        T.dlink((self, "composite"), (slides, "pdf"))
        self.update_composite()
        return slides
    
    @T.observe("children")
    def _on_children(self, change):
        self.update_composite()
    
    def update_composite(self):
        tree = E.fromstring("""<mxfile version="13.3.4"></mxfile>""")
        with tempfile.TemporaryDirectory() as td:
            tdp = P.Path(td)
            merger = PyPDF2.PdfFileMerger()
            for i, child in enumerate(self.children):
                next_pdf = (tdp / f"doc-{i}.pdf")
                xml = E.fromstring(child.xml)
                tag = tree.tag
                if tag == "mxfile":
                    [tree.append(d) for d in xml.xpath("//diagram")]
                elif tag == "mxGraphModel":
                    d = E.Element("diagram")
                    d.append(xml)
                    tree.append(d)
                wrote = next_pdf.write_bytes(base64.b64decode(child.pdf.encode("utf-8")))
                if wrote:
                    merger.append(PyPDF2.PdfFileReader(str(next_pdf)))
            output_pdf = tdp / "output.pdf"
            final_pdf = tdp / "final.pdf"
            merger.write(str(output_pdf))
            self.composite_xml = E.tostring(tree).decode("utf-8")
            final = PyPDF2.PdfFileWriter()
            final.appendPagesFromReader(PyPDF2.PdfFileReader(str(output_pdf), "rb"))
            final.addAttachment("drawing.drawio", self.composite_xml.encode("utf-8"))
            with final_pdf.open("wb") as fpt:
                final.write(fpt)
            self.composite = base64.b64encode(final_pdf.read_bytes()).decode("utf-8")

In [ ]:
new_deck = Deck(deck.children)

In [ ]:
new_deck.preview